# Week 3B: Data Filtering and Grouping Operations

## ISM6251: Machine Learning for Business Applications

### Learning Objectives
By the end of this notebook, you will be able to:
- Apply advanced filtering techniques to extract relevant data subsets
- Use grouping operations for aggregation and analysis
- Combine filtering and grouping for complex data transformations
- Handle real-world data manipulation scenarios

---

## Part 1: Data Filtering Techniques

Filtering is one of the most fundamental operations in data preparation. It allows us to:
- Focus on relevant subsets of data
- Remove outliers and anomalies
- Create training/validation splits
- Prepare data for specific analyses

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

# Set style for visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Libraries loaded successfully!")

### 1.1 Creating Sample Dataset

Let's create a comprehensive sales dataset to demonstrate filtering and grouping operations:

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate sample sales data
n_records = 1000

# Create date range
start_date = datetime(2023, 1, 1)
dates = [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(n_records)]

# Create sample data
sales_data = pd.DataFrame({
    'transaction_id': range(1001, 1001 + n_records),
    'date': dates,
    'customer_id': np.random.choice(range(100, 200), n_records),
    'product': np.random.choice(['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard', 'Mouse'], n_records),
    'category': np.random.choice(['Electronics', 'Accessories'], n_records),
    'quantity': np.random.randint(1, 6, n_records),
    'unit_price': np.random.uniform(50, 2000, n_records),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n_records),
    'sales_rep': np.random.choice(['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'], n_records),
    'customer_type': np.random.choice(['Regular', 'Premium', 'New'], n_records, p=[0.5, 0.3, 0.2])
})

# Calculate total amount
sales_data['total_amount'] = sales_data['quantity'] * sales_data['unit_price']

# Add some missing values for demonstration
missing_indices = np.random.choice(sales_data.index, 50, replace=False)
sales_data.loc[missing_indices, 'customer_type'] = np.nan

# Sort by date
sales_data = sales_data.sort_values('date').reset_index(drop=True)

print(f"Dataset created with {len(sales_data)} records")
print(f"\nDataset shape: {sales_data.shape}")
print(f"\nFirst 5 records:")
sales_data.head()

### 1.2 Boolean Indexing

Boolean indexing is the foundation of filtering in pandas. It creates a mask of True/False values:

In [ ]:
# Simple boolean condition
high_value_mask = sales_data['total_amount'] > 5000

print("Boolean mask (first 10 values):")
print(high_value_mask.head(10))
print(f"\nNumber of True values: {high_value_mask.sum()}")
print(f"Percentage of high-value transactions: {high_value_mask.mean()*100:.2f}%")

In [ ]:
# Apply the boolean mask
high_value_sales = sales_data[high_value_mask]

print(f"Original dataset: {len(sales_data)} records")
print(f"Filtered dataset: {len(high_value_sales)} records")
print(f"\nSample of high-value transactions:")
high_value_sales.head()

### 1.3 Multiple Conditions

Combine conditions using `&` (and), `|` (or), and `~` (not):

In [ ]:
# Complex filtering with multiple conditions
filtered_data = sales_data[
    (sales_data['region'] == 'North') & 
    (sales_data['total_amount'] > 3000) & 
    (sales_data['product'].isin(['Laptop', 'Phone']))
]

print(f"Filtered records: {len(filtered_data)}")
print(f"\nAverage transaction value: ${filtered_data['total_amount'].mean():.2f}")
print(f"\nProduct distribution:")
print(filtered_data['product'].value_counts())

### 1.4 Query Method

The `.query()` method provides a more readable syntax for filtering:

In [ ]:
# Using query method for filtering
premium_electronics = sales_data.query(
    "customer_type == 'Premium' and category == 'Electronics' and total_amount > 2000"
)

print(f"Premium electronics transactions: {len(premium_electronics)}")
print(f"\nTop 5 transactions by amount:")
premium_electronics.nlargest(5, 'total_amount')[['date', 'product', 'total_amount', 'sales_rep']]

### 1.5 Advanced Filtering Techniques

In [ ]:
# Filter by date range
q2_start = pd.Timestamp('2023-04-01')
q2_end = pd.Timestamp('2023-06-30')

q2_sales = sales_data[
    (sales_data['date'] >= q2_start) & 
    (sales_data['date'] <= q2_end)
]

print(f"Q2 2023 Sales: {len(q2_sales)} transactions")
print(f"Total Q2 Revenue: ${q2_sales['total_amount'].sum():,.2f}")

In [ ]:
# Filter using string methods
phone_related = sales_data[
    sales_data['product'].str.contains('Phone', case=False, na=False)
]

print(f"Phone-related products: {len(phone_related)} transactions")
print(f"Unique products: {phone_related['product'].unique()}")

In [ ]:
# Filter by percentile
percentile_75 = sales_data['total_amount'].quantile(0.75)
top_quartile = sales_data[sales_data['total_amount'] > percentile_75]

print(f"75th percentile threshold: ${percentile_75:.2f}")
print(f"Transactions in top quartile: {len(top_quartile)}")
print(f"\nTop quartile statistics:")
print(top_quartile['total_amount'].describe())

### Practice Exercise 1: Filtering

Complete the following filtering tasks:

In [ ]:
# TODO: Find all transactions from the 'West' region in Q3 2023 (July-September)
# Your code here:


# TODO: Find transactions where the sales rep is either 'Alice' or 'Bob' 
# and the quantity is greater than 3
# Your code here:


# TODO: Find the top 10% of transactions by total_amount
# Your code here:


## Part 2: Grouping and Aggregation

Grouping allows us to:
- Summarize data by categories
- Calculate statistics for different segments
- Identify patterns and trends
- Prepare data for visualization

### 2.1 Basic Grouping

In [ ]:
# Group by single column
region_summary = sales_data.groupby('region')['total_amount'].sum()

print("Total sales by region:")
print(region_summary.sort_values(ascending=False))

# Visualize
plt.figure(figsize=(10, 6))
region_summary.sort_values().plot(kind='barh')
plt.title('Total Sales by Region')
plt.xlabel('Total Sales ($)')
plt.tight_layout()
plt.show()

### 2.2 Multiple Aggregations

In [ ]:
# Multiple aggregation functions
product_stats = sales_data.groupby('product')['total_amount'].agg([
    'count',
    'mean',
    'median',
    'std',
    'min',
    'max',
    'sum'
]).round(2)

product_stats.columns = ['Transactions', 'Avg_Amount', 'Median_Amount', 
                         'Std_Dev', 'Min_Amount', 'Max_Amount', 'Total_Sales']

print("Product Performance Statistics:")
product_stats.sort_values('Total_Sales', ascending=False)

### 2.3 Grouping by Multiple Columns

In [ ]:
# Group by multiple columns
region_product = sales_data.groupby(['region', 'product']).agg({
    'total_amount': ['sum', 'mean'],
    'quantity': 'sum',
    'transaction_id': 'count'
}).round(2)

# Flatten column names
region_product.columns = ['_'.join(col).strip() for col in region_product.columns]
region_product = region_product.rename(columns={'transaction_id_count': 'num_transactions'})

print("Top 10 Region-Product Combinations by Total Sales:")
region_product.sort_values('total_amount_sum', ascending=False).head(10)

### 2.4 Custom Aggregation Functions

In [ ]:
# Define custom aggregation functions
def revenue_range(x):
    return x.max() - x.min()

def coefficient_of_variation(x):
    return (x.std() / x.mean()) * 100 if x.mean() > 0 else 0

# Apply custom functions
sales_rep_analysis = sales_data.groupby('sales_rep')['total_amount'].agg([
    'count',
    'sum',
    'mean',
    ('range', revenue_range),
    ('cv', coefficient_of_variation)
]).round(2)

sales_rep_analysis.columns = ['Num_Sales', 'Total_Revenue', 'Avg_Sale', 
                              'Revenue_Range', 'Coefficient_Variation']

print("Sales Representative Performance Analysis:")
print(sales_rep_analysis.sort_values('Total_Revenue', ascending=False))
print("\nNote: Coefficient of Variation measures consistency (lower = more consistent)")

### 2.5 Transform and Apply Operations

In [ ]:
# Add group-level statistics to original dataframe
sales_data['region_avg'] = sales_data.groupby('region')['total_amount'].transform('mean')
sales_data['above_region_avg'] = sales_data['total_amount'] > sales_data['region_avg']

# Calculate percentage above regional average
pct_above = sales_data.groupby('region')['above_region_avg'].mean() * 100

print("Percentage of transactions above regional average:")
for region, pct in pct_above.items():
    print(f"{region}: {pct:.1f}%")

In [ ]:
# Normalize sales within each product category
def normalize_group(x):
    return (x - x.mean()) / x.std() if x.std() > 0 else 0

sales_data['normalized_amount'] = sales_data.groupby('product')['total_amount'].transform(normalize_group)

# Find outliers (transactions with normalized amount > 2 or < -2)
outliers = sales_data[abs(sales_data['normalized_amount']) > 2]

print(f"Number of outlier transactions: {len(outliers)}")
print(f"\nOutlier distribution by product:")
print(outliers['product'].value_counts())

### 2.6 Time-based Grouping

In [ ]:
# Add time components
sales_data['month'] = sales_data['date'].dt.month
sales_data['quarter'] = sales_data['date'].dt.quarter
sales_data['day_of_week'] = sales_data['date'].dt.dayofweek
sales_data['week_of_year'] = sales_data['date'].dt.isocalendar().week

# Monthly sales trend
monthly_sales = sales_data.groupby('month').agg({
    'total_amount': 'sum',
    'transaction_id': 'count',
    'quantity': 'sum'
}).round(2)

monthly_sales.columns = ['Total_Revenue', 'Num_Transactions', 'Total_Units']
monthly_sales['Avg_Transaction'] = monthly_sales['Total_Revenue'] / monthly_sales['Num_Transactions']

# Visualize monthly trend
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

monthly_sales['Total_Revenue'].plot(kind='bar', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('Monthly Revenue')
axes[0, 0].set_xlabel('Month')
axes[0, 0].set_ylabel('Revenue ($)')

monthly_sales['Num_Transactions'].plot(kind='bar', ax=axes[0, 1], color='lightgreen')
axes[0, 1].set_title('Monthly Transaction Count')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Number of Transactions')

monthly_sales['Avg_Transaction'].plot(kind='line', ax=axes[1, 0], marker='o', color='coral')
axes[1, 0].set_title('Average Transaction Value by Month')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Avg Transaction ($)')
axes[1, 0].grid(True, alpha=0.3)

# Day of week analysis
dow_sales = sales_data.groupby('day_of_week')['total_amount'].sum()
dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dow_sales.index = dow_labels
dow_sales.plot(kind='bar', ax=axes[1, 1], color='plum')
axes[1, 1].set_title('Sales by Day of Week')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Total Sales ($)')

plt.tight_layout()
plt.show()

print("Monthly Sales Summary:")
print(monthly_sales)

### Practice Exercise 2: Grouping

Complete the following grouping tasks:

In [ ]:
# TODO: Calculate the average quantity sold by each sales rep for each product
# Your code here:


# TODO: Find the best performing day of the week for each region
# Your code here:


# TODO: Calculate the running total of sales for each sales rep over time
# Hint: Use groupby with cumsum()
# Your code here:


## Part 3: Combining Filtering and Grouping

Real-world analysis often requires combining filtering and grouping operations:

### 3.1 Filter Then Group

In [ ]:
# Filter for premium customers, then analyze by region
premium_only = sales_data[sales_data['customer_type'] == 'Premium']

premium_by_region = premium_only.groupby('region').agg({
    'total_amount': ['sum', 'mean', 'count'],
    'customer_id': 'nunique'
}).round(2)

premium_by_region.columns = ['Total_Revenue', 'Avg_Transaction', 
                             'Num_Transactions', 'Unique_Customers']

print("Premium Customer Analysis by Region:")
print(premium_by_region)
print(f"\nTotal Premium Revenue: ${premium_by_region['Total_Revenue'].sum():,.2f}")

### 3.2 Group Then Filter

In [ ]:
# Calculate customer lifetime value and filter top customers
customer_ltv = sales_data.groupby('customer_id').agg({
    'total_amount': 'sum',
    'transaction_id': 'count',
    'date': ['min', 'max']
}).round(2)

customer_ltv.columns = ['Total_Spent', 'Num_Purchases', 'First_Purchase', 'Last_Purchase']
customer_ltv['Days_Active'] = (customer_ltv['Last_Purchase'] - customer_ltv['First_Purchase']).dt.days
customer_ltv['Avg_Purchase'] = customer_ltv['Total_Spent'] / customer_ltv['Num_Purchases']

# Filter top 20% of customers by total spent
threshold = customer_ltv['Total_Spent'].quantile(0.8)
top_customers = customer_ltv[customer_ltv['Total_Spent'] > threshold]

print(f"Top 20% Customer Threshold: ${threshold:.2f}")
print(f"Number of top customers: {len(top_customers)}")
print(f"\nTop 10 Customers by Lifetime Value:")
print(top_customers.nlargest(10, 'Total_Spent')[['Total_Spent', 'Num_Purchases', 'Avg_Purchase']])

### 3.3 Complex Analysis Pipeline

In [ ]:
# Complex analysis: Product performance by customer segment in Q2

# Step 1: Filter for Q2
q2_data = sales_data[
    (sales_data['date'] >= '2023-04-01') & 
    (sales_data['date'] <= '2023-06-30')
].copy()

# Step 2: Clean customer_type (fill NaN with 'Unknown')
q2_data['customer_type'] = q2_data['customer_type'].fillna('Unknown')

# Step 3: Group by customer type and product
segment_product_analysis = q2_data.groupby(['customer_type', 'product']).agg({
    'total_amount': ['sum', 'mean', 'count'],
    'quantity': 'sum'
}).round(2)

segment_product_analysis.columns = ['Revenue', 'Avg_Transaction', 'Num_Orders', 'Units_Sold']
segment_product_analysis = segment_product_analysis.reset_index()

# Step 4: Calculate metrics
segment_product_analysis['Revenue_per_Unit'] = (
    segment_product_analysis['Revenue'] / segment_product_analysis['Units_Sold']
).round(2)

# Step 5: Find top product for each segment
top_products = segment_product_analysis.loc[
    segment_product_analysis.groupby('customer_type')['Revenue'].idxmax()
]

print("Top Product by Customer Segment (Q2 2023):")
print("="*60)
for _, row in top_products.iterrows():
    print(f"\n{row['customer_type']} Customers:")
    print(f"  Top Product: {row['product']}")
    print(f"  Revenue: ${row['Revenue']:,.2f}")
    print(f"  Orders: {int(row['Num_Orders'])}")
    print(f"  Avg Transaction: ${row['Avg_Transaction']:.2f}")

### 3.4 Pivot Tables for Analysis

In [ ]:
# Create pivot table for region-product analysis
pivot_revenue = pd.pivot_table(
    sales_data,
    values='total_amount',
    index='product',
    columns='region',
    aggfunc='sum',
    fill_value=0
).round(2)

# Add totals
pivot_revenue['Total'] = pivot_revenue.sum(axis=1)
pivot_revenue.loc['Total'] = pivot_revenue.sum()

print("Revenue by Product and Region:")
print(pivot_revenue)

# Visualize as heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(
    pivot_revenue.iloc[:-1, :-1],  # Exclude totals for heatmap
    annot=True,
    fmt='.0f',
    cmap='YlOrRd',
    cbar_kws={'label': 'Revenue ($)'}
)
plt.title('Revenue Heatmap: Products vs Regions')
plt.tight_layout()
plt.show()

### Practice Exercise 3: Combined Operations

Complete these combined filtering and grouping tasks:

In [ ]:
# TODO: Find the best performing sales rep in each region for Electronics only
# Best = highest total revenue
# Your code here:


# TODO: Calculate the month-over-month growth rate for each product
# Hint: Use groupby, then calculate percentage change
# Your code here:


# TODO: Identify customers who have purchased from all 4 regions
# Your code here:


## Part 4: Performance Optimization

When working with large datasets, optimization becomes crucial:

### 4.1 Efficient Filtering

In [ ]:
import time

# Create a larger dataset for performance testing
large_data = pd.concat([sales_data] * 100, ignore_index=True)
print(f"Large dataset size: {len(large_data):,} records")

# Method 1: Chained filtering (inefficient)
start = time.time()
result1 = large_data[large_data['region'] == 'North']
result1 = result1[result1['product'] == 'Laptop']
result1 = result1[result1['total_amount'] > 1000]
time1 = time.time() - start

# Method 2: Single boolean mask (efficient)
start = time.time()
result2 = large_data[
    (large_data['region'] == 'North') & 
    (large_data['product'] == 'Laptop') & 
    (large_data['total_amount'] > 1000)
]
time2 = time.time() - start

# Method 3: Query method
start = time.time()
result3 = large_data.query(
    "region == 'North' and product == 'Laptop' and total_amount > 1000"
)
time3 = time.time() - start

print(f"\nPerformance Comparison:")
print(f"Chained filtering: {time1:.4f} seconds")
print(f"Single mask: {time2:.4f} seconds")
print(f"Query method: {time3:.4f} seconds")
print(f"\nSpeedup: {time1/time2:.2f}x for single mask vs chained")

### 4.2 Efficient Grouping

In [ ]:
# Optimize data types before grouping
optimized_data = large_data.copy()

# Convert to categorical for repeated values
categorical_cols = ['region', 'product', 'category', 'sales_rep', 'customer_type']
for col in categorical_cols:
    optimized_data[col] = optimized_data[col].astype('category')

# Compare memory usage
original_memory = large_data.memory_usage(deep=True).sum() / 1024**2
optimized_memory = optimized_data.memory_usage(deep=True).sum() / 1024**2

print(f"Original memory: {original_memory:.2f} MB")
print(f"Optimized memory: {optimized_memory:.2f} MB")
print(f"Memory reduction: {(1 - optimized_memory/original_memory)*100:.1f}%")

# Compare grouping performance
start = time.time()
result1 = large_data.groupby(['region', 'product'])['total_amount'].sum()
time1 = time.time() - start

start = time.time()
result2 = optimized_data.groupby(['region', 'product'])['total_amount'].sum()
time2 = time.time() - start

print(f"\nGrouping performance:")
print(f"Original data: {time1:.4f} seconds")
print(f"Optimized data: {time2:.4f} seconds")
print(f"Speedup: {time1/time2:.2f}x")

## Summary and Best Practices

### Filtering Best Practices:
1. **Use vectorized operations** instead of loops
2. **Combine conditions** in a single mask when possible
3. **Use `.isin()` for multiple value checks**
4. **Consider `.query()` for complex conditions**
5. **Filter early** in your data pipeline

### Grouping Best Practices:
1. **Use appropriate aggregation functions**
2. **Consider memory usage** with large groups
3. **Use `.agg()` for multiple aggregations**
4. **Optimize data types** before grouping
5. **Use pivot tables** for cross-tabulation

### Performance Tips:
1. **Convert to categorical** for repeated strings
2. **Use appropriate data types** (int32 vs int64)
3. **Filter before grouping** when possible
4. **Cache intermediate results**
5. **Consider chunking** for very large datasets

## Final Exercise: Real-World Scenario

You're tasked with identifying high-value customer segments for a targeted marketing campaign:

In [ ]:
# TODO: Complete this analysis

# 1. Identify customers who:
#    - Have made at least 5 purchases
#    - Have average transaction > $2000
#    - Have purchased in at least 2 different quarters

# 2. For these high-value customers, calculate:
#    - Their preferred product category
#    - Their most active region
#    - Their total lifetime value

# 3. Create a summary report showing:
#    - Number of high-value customers
#    - Their contribution to total revenue
#    - Recommendations for marketing campaign

# Your code here:


## Conclusion

In this notebook, we've covered:
- **Advanced filtering techniques** using boolean indexing, query methods, and complex conditions
- **Grouping and aggregation** operations for data summarization
- **Combining operations** for complex data analysis
- **Performance optimization** strategies for large datasets

These skills form the foundation for effective data manipulation and are essential for preparing data for machine learning models.

### Next Steps:
- Practice with your own datasets
- Explore more advanced pandas functionality
- Learn about SQL for database operations
- Apply these techniques to real business problems